# TensorFlow 2 quickstart

Import TensorFlow into your program:

In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, BatchNormalization, Dropout, GlobalMaxPooling2D
from tensorflow.keras import Input, datasets, layers, models, Model
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.6.0


# MNIST Handwritten Digit Classification Dataset

Load and prepare the [MNIST dataset](http://yann.lecun.com/exdb/mnist/).

In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

In [3]:
x_train.shape, x_test.shape

((60000, 28, 28), (10000, 28, 28))

We have 60000 samples for training dataset and 10000 samples for testing dataset. Lets take 10000 samples from training data and keep them for validation dataset.

In [4]:
x_train, y_train = x_train[:-10000], y_train[:-10000]
x_val, y_val = x_train[-10000:], y_train[-10000:]

Check the data type of the dataset

In [5]:
type(x_train), type(y_train)

(numpy.ndarray, numpy.ndarray)

Normalizing the data to range [0,1]

In [6]:
x_train, x_val, x_test = x_train / 255.0, x_val / 225.0, x_test / 255.0

Inspecting the shape of the data

In [7]:
x_train.shape, x_val.shape, x_test.shape

((50000, 28, 28), (10000, 28, 28), (10000, 28, 28))

Adding one more dimension for channel.

In [8]:
x_train = x_train[:,:,:,tf.newaxis].astype("float32")
x_val = x_val[:,:,:,tf.newaxis].astype("float32")
x_test = x_test[:,:,:,tf.newaxis].astype("float32")

In [9]:
x_train.shape, x_val.shape, x_test.shape

((50000, 28, 28, 1), (10000, 28, 28, 1), (10000, 28, 28, 1))

# Comparable Modelling Strategies in TensorFlow 2
In TF.Keras there are basically three-way we can define a neural network, namely
* Sequential API
* Functional API
* Model Subclassing API

Among them, Sequential API is the easiest way to implement but comes with certain limitations. For example, with this API, we can't create a model that shares feature information to another layer except to its subsequent layer. In addition, multiple input and output are not possible to implement either.

In this point, Functional API does solve these issues greatly. A model like Inception or ResNet is feasible to implement in Functional API. But often deep learning researcher wants to have more control over every nuance of the network and on the training pipelines and that's exactly what Model Subclassing API serves. Model Sub-Classing is a fully customizable way to implement the feed-forward mechanism for our custom-designed deep neural network in an object-oriented fashion.

Let's create a very basic neural network using these three API. It will be the same neural architecture and will see what are the implementation differences. This of course will not demonstrate the full potential, especially for Functional and Model Sub-Classing API. The architecture will be as follows:

`Input -> Conv -> MaxPool -> BN -> Conv -> BN -> Droput -> GAP -> Dense`

In [10]:
input_dim = (28, 28, 1)
output_dim = (10)

Build the `tf.keras` model using the Keras model [Sequential API](https://www.tensorflow.org/api_docs/python/tf/keras/Sequential):

In [11]:
seq_model = tf.keras.Sequential()
# Declare input Shape 
seq_model.add(Input(shape=input_dim))

# Block 1
seq_model.add(Conv2D(32, 3, strides=2, activation="relu"))
seq_model.add(MaxPooling2D(3))
seq_model.add(BatchNormalization())

# Block 2
seq_model.add(Conv2D(64, 3, activation="relu"))
seq_model.add(BatchNormalization())
seq_model.add(Dropout(0.3))

# Now that we apply global max pooling.
seq_model.add(GlobalMaxPooling2D())

# Finally, we add a classification layer.
seq_model.add(Dense(output_dim))

In [12]:
seq_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 13, 13, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 4, 4, 32)          0         
_________________________________________________________________
batch_normalization (BatchNo (None, 4, 4, 32)          128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 2, 2, 64)          18496     
_________________________________________________________________
batch_normalization_1 (Batch (None, 2, 2, 64)          256       
_________________________________________________________________
dropout (Dropout)            (None, 2, 2, 64)          0         
_________________________________________________________________
global_max_pooling2d (Global (None, 64)                0

Build the `tf.keras` model using the Keras model [Functional API](https://www.tensorflow.org/guide/keras/functional)

In [13]:
# Declare input shape 
input = tf.keras.Input(shape=(input_dim))

# Block 1
x = Conv2D(32, 3, strides=2, activation="relu")(input)
x = MaxPooling2D(3)(x)
x = BatchNormalization()(x)

# Block 2
x = Conv2D(64, 3, activation="relu")(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)

# Now that we apply global max pooling.
gap = GlobalMaxPooling2D()(x)

# Finally, we add a classification layer.
output = Dense(output_dim)(gap)

# bind all
func_model = Model(input, output)

In [14]:
func_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 13, 13, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 4, 4, 32)          0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 4, 4, 32)          128       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 2, 2, 64)          18496     
_________________________________________________________________
batch_normalization_3 (Batch (None, 2, 2, 64)          256       
_________________________________________________________________
dropout_1 (Dropout)          (None, 2, 2, 64)          0     

Build the `tf.keras` model using the Keras [model subclassing API](https://www.tensorflow.org/guide/keras/custom_layers_and_models):

In [15]:
class ModelSubClassing(tf.keras.Model):
    def __init__(self, num_classes):
        super(ModelSubClassing, self).__init__()
        # Define all layers in init
        # Layer of Block 1
        self.conv1 = Conv2D(32, 3, strides=2, activation="relu")
        self.max1  = MaxPooling2D(3)
        self.bn1   = BatchNormalization()

        # Layer of Block 2
        self.conv2 = tf.keras.layers.Conv2D(64, 3, activation="relu")
        self.bn2   = tf.keras.layers.BatchNormalization()
        self.drop  = tf.keras.layers.Dropout(0.3)

        # GAP, followed by Classifier
        self.gap   = tf.keras.layers.GlobalAveragePooling2D()
        self.dense = tf.keras.layers.Dense(num_classes)


    def call(self, input_tensor, training=False):
        # forward pass: block 1 
        x = self.conv1(input_tensor)
        x = self.max1(x)
        x = self.bn1(x)

        # forward pass: block 2 
        x = self.conv2(x)
        x = self.bn2(x)

        # droput followed by gap and classifier
        x = self.drop(x)
        x = self.gap(x)
        return self.dense(x)

In [16]:
sub_model = ModelSubClassing(output_dim)
sub_model.build(input_shape = (None, 24, 24, 1))
sub_model.call(Input(shape=input_dim))
sub_model.summary()

Model: "model_sub_classing"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 13, 13, 32)        320       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 4, 4, 32)          0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 4, 4, 32)          128       
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 2, 2, 64)          18496     
_________________________________________________________________
batch_normalization_5 (Batch (None, 2, 2, 64)          256       
_________________________________________________________________
dropout_2 (Dropout)          (None, 2, 2, 64)          0         
_________________________________________________________________
global_average_pooling2d (Gl (None, 64)         

# Training - The standard Method

+ model.compile()
+ model.fit()
+ model.evaluate()
+ model.predict()

In [17]:
# compile 
print('Sequential API')
seq_model.compile(
          loss      = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
          metrics   = tf.keras.metrics.SparseCategoricalAccuracy(),
          optimizer = tf.keras.optimizers.Adam())
# fit 
seq_model.fit(x_train, y_train, batch_size=32, epochs=5)

# evaluate
print("Evaluating on validation data")
seq_model.evaluate(x_val, y_val)

#########################################################################################

# compile 
print('\nFunctional API')
func_model.compile(
          loss      = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
          metrics   = tf.keras.metrics.SparseCategoricalAccuracy(),
          optimizer = tf.keras.optimizers.Adam())
# fit 
func_model.fit(x_train, y_train, batch_size=32, epochs=5)

# evaluate
print("Evaluating on validation data")
func_model.evaluate(x_val, y_val)

##########################################################################################

# compile 
print('\nModel Sub-Classing API')
sub_model = ModelSubClassing(10)
sub_model.compile(
          loss      = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
          metrics   = tf.keras.metrics.SparseCategoricalAccuracy(),
          optimizer = tf.keras.optimizers.Adam())
# fit 
sub_model.fit(x_train, y_train, batch_size=32, epochs=5)

# evaluate
print("Evaluating on validation data")
sub_model.evaluate(x_val, y_val)

print("\nFinished Training models")

Sequential API
Epoch 1/5
1563/1563 [==============================] - 8s 4ms/step - loss: 0.3196 - sparse_categorical_accuracy: 0.9019
Epoch 2/5
1563/1563 [==============================] - 7s 4ms/step - loss: 0.1326 - sparse_categorical_accuracy: 0.9587
Epoch 3/5
1563/1563 [==============================] - 7s 4ms/step - loss: 0.1076 - sparse_categorical_accuracy: 0.9661
Epoch 4/5
1563/1563 [==============================] - 7s 4ms/step - loss: 0.0945 - sparse_categorical_accuracy: 0.9701
Epoch 5/5
1563/1563 [==============================] - 6s 4ms/step - loss: 0.0836 - sparse_categorical_accuracy: 0.9737
Evaluating on validation data
313/313 [==============================] - 1s 3ms/step - loss: 0.0777 - sparse_categorical_accuracy: 0.9732

Functional API
Epoch 1/5
1563/1563 [==============================] - 7s 4ms/step - loss: 0.2959 - sparse_categorical_accuracy: 0.9078
Epoch 2/5
1563/1563 [==============================] - 7s 4ms/step - loss: 0.1340 - sparse_categorical_accuracy

model.predict() is the function we use to make single prediction

# Custom Training

For custom training we follow these steps:

+ Set optimizer, loss functions and metrics.

+ Run a loop for number of epochs

  + Run a nested loop for each batch of each epoch:
  + Work with `tf.GradientTape()` to calculate and record loss and to conduct backpropagation.
  + Run the optimizer
  + Calculate, record and print out the metric results.


# Convert Numpy array to TF Dataset Object


In [18]:
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(buffer_size = 50000).batch(32)
val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val)).batch(32)

Choose an optimizer and loss function for training: 

In [19]:
loss, optimizer = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), tf.keras.optimizers.Adam()

Select metrics to measure the loss and the accuracy of the model. These metrics accumulate the values over epochs and then print the overall result.

In [20]:
train_acc_metric = tf.keras.metrics.SparseCategoricalAccuracy()
val_acc_metric = tf.keras.metrics.SparseCategoricalAccuracy()

In [21]:
import time

epochs = 5
for epoch in range(epochs):
    print(f"\nStart of epoch {epoch}")
    start_time = time.time()

    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            logits = sub_model(x_batch_train, training=True)  # Logits for this minibatch
            loss_value = loss(y_batch_train, logits)
        grads = tape.gradient(loss_value, sub_model.trainable_weights)
        optimizer.apply_gradients(zip(grads, sub_model.trainable_weights))
        train_acc_metric.update_state(y_batch_train, logits)

        if step % 200 == 0:
            print(f"Training loss (for one batch) at step {step}: {round(float(loss_value), 4)}")
            print(f"Seen so far: {((step + 1) * 32)} samples")

    train_acc = train_acc_metric.result()
    print(f"Training accuracy over epoch: {round(float(train_acc), 4)}")
    train_acc_metric.reset_states()

    for x_batch_val, y_batch_val in val_dataset:
        val_logits = sub_model(x_batch_val, training=False)
        val_acc_metric.update_state(y_batch_val, val_logits)
    val_acc = val_acc_metric.result()
    val_acc_metric.reset_states()
    print(f"Validation accuracy:{round(float(train_acc), 4)}")
    print(f"Time taken:{(time.time() - start_time)}")


Start of epoch 0
Training loss (for one batch) at step 0: 0.0921
Seen so far: 32 samples
Training loss (for one batch) at step 200: 0.0521
Seen so far: 6432 samples
Training loss (for one batch) at step 400: 0.3219
Seen so far: 12832 samples
Training loss (for one batch) at step 600: 0.0439
Seen so far: 19232 samples
Training loss (for one batch) at step 800: 0.0331
Seen so far: 25632 samples
Training loss (for one batch) at step 1000: 0.1172
Seen so far: 32032 samples
Training loss (for one batch) at step 1200: 0.0103
Seen so far: 38432 samples
Training loss (for one batch) at step 1400: 0.013
Seen so far: 44832 samples
Training accuracy over epoch: 0.9755
Validation accuracy:0.9755
Time taken:32.84599494934082

Start of epoch 1
Training loss (for one batch) at step 0: 0.0129
Seen so far: 32 samples
Training loss (for one batch) at step 200: 0.2852
Seen so far: 6432 samples
Training loss (for one batch) at step 400: 0.0655
Seen so far: 12832 samples
Training loss (for one batch) at s

# Speeding-up training step with `tf.function`

The default runtime in TensorFlow 2 is eager execution. As such, our training loop above executes eagerly.

This is great for debugging, but graph compilation has a definite performance advantage. Describing the computation as a static graph enables the framework to apply global performance optimizations. This is impossible when the framework is constrained to greedly execute one operation after another, with no knowledge of what comes next.

You can compile into a static graph any function that takes tensors as input.  
Just add a @tf.function decorator on it, like this:

In [22]:
@tf.function
def train_step(x, y):
    with tf.GradientTape() as tape:
        logits = sub_model(x, training=True)
        loss_value = loss(y, logits)
    grads = tape.gradient(loss_value, sub_model.trainable_weights)
    optimizer.apply_gradients(zip(grads, sub_model.trainable_weights))
    train_acc_metric.update_state(y, logits)
    return loss_value

In [23]:
@tf.function
def val_step(x, y):
    val_logits = sub_model(x, training=False)
    val_acc_metric.update_state(y, val_logits)

In [24]:
import time

epochs = 5
for epoch in range(epochs):
    print(f"\nStart of epoch {epoch}")
    start_time = time.time()

    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        loss_value = train_step(x_batch_train, y_batch_train)
  
        if step % 200 == 0:
            print(f"Training loss (for one batch) at step {step}: {round(float(loss_value), 4)}")
            print(f"Seen so far: {((step + 1) * 32)} samples")

    train_acc = train_acc_metric.result()
    print(f"Training accuracy over epoch: {round(float(train_acc), 4)}")
    train_acc_metric.reset_states()

    for x_batch_val, y_batch_val in val_dataset:
       val_step(x_batch_val, y_batch_val)

    val_acc = val_acc_metric.result()
    val_acc_metric.reset_states()
    print(f"Validation accuracy:{round(float(train_acc), 4)}")
    print(f"Time taken:{(time.time() - start_time)}")


Start of epoch 0
Training loss (for one batch) at step 0: 0.1273
Seen so far: 32 samples
Training loss (for one batch) at step 200: 0.1019
Seen so far: 6432 samples
Training loss (for one batch) at step 400: 0.0085
Seen so far: 12832 samples
Training loss (for one batch) at step 600: 0.1863
Seen so far: 19232 samples
Training loss (for one batch) at step 800: 0.0096
Seen so far: 25632 samples
Training loss (for one batch) at step 1000: 0.0799
Seen so far: 32032 samples
Training loss (for one batch) at step 1200: 0.0116
Seen so far: 38432 samples
Training loss (for one batch) at step 1400: 0.0339
Seen so far: 44832 samples
Training accuracy over epoch: 0.982
Validation accuracy:0.982
Time taken:5.274996280670166

Start of epoch 1
Training loss (for one batch) at step 0: 0.0371
Seen so far: 32 samples
Training loss (for one batch) at step 200: 0.0373
Seen so far: 6432 samples
Training loss (for one batch) at step 400: 0.0076
Seen so far: 12832 samples
Training loss (for one batch) at st